# Crypto bot metrics evaluation:

## Test data extraction from Binance

In [109]:
from src.get_historical_data import get_historical_data

LOOKBACK = 200
COIN = 'SOLUSDT'

df = get_historical_data(coin=COIN,lookback=LOOKBACK)
df

,date,SOLUSDT
0,2024-08-19,144.29
1,2024-08-20,142.27
2,2024-08-21,142.79
3,2024-08-22,143.22
4,2024-08-23,153.26
...,...,...
195,2025-03-02,178.71
196,2025-03-03,142.16
197,2025-03-04,144.79
198,2025-03-05,146.28


### Plotting data results:

In [110]:
import plotly.express as px
px.line(df, x='date', y=COIN)

## Extracting exchange interest from YFinance

In [111]:
from src.get_treasury_rate import get_treasury_rate

df2 = get_treasury_rate(lookback=LOOKBACK)
df2

[*********************100%***********************]  1 of 1 completed


,date,treasury_rate
0,2024-08-19,5.063
1,2024-08-20,5.033
2,2024-08-21,5.005
3,2024-08-22,5.025
4,2024-08-23,5.000
...,...,...
132,2025-02-28,4.193
133,2025-03-03,4.193
134,2025-03-04,4.195
135,2025-03-05,4.205


## Testing metrics

In [112]:
from src.crypto_metrics import CryptoMetrics
metrics = CryptoMetrics(lookback=LOOKBACK)
rsi = metrics.calculate_rsi(df=df, coin=COIN)

### RSI:

In [113]:
fig = px.line(rsi, x='date', y=COIN, title=f'{COIN} RSI', labels={f'{COIN}': f'{COIN} Price', 'date': 'Date'})

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=rsi[rsi[f'rsi_buy_{COIN}'] == 1]['date'], 
                y=rsi[rsi[f'rsi_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='RSI Buy')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=rsi[rsi[f'rsi_sell_{COIN}'] == 1]['date'], 
                y=rsi[rsi[f'rsi_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='RSI Sell')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Treasury Rate correlation:

In [114]:
from importlib import reload
import src.crypto_metrics
reload(src.crypto_metrics)
from src.crypto_metrics import CryptoMetrics
treasury_corr = metrics.calculate_corr_treasury(df=df, coin=COIN)
treasury_corr

[*********************100%***********************]  1 of 1 completed


,date,SOLUSDT,treasury_rate,treasury_corr_SOLUSDT,treasury_corr_buy_SOLUSDT,treasury_corr_sell_SOLUSDT
0,2024-08-19,144.29,5.063,NaN,False,False
1,2024-08-20,142.27,5.033,NaN,False,False
2,2024-08-21,142.79,5.005,NaN,False,False
3,2024-08-22,143.22,5.025,NaN,False,False
4,2024-08-23,153.26,5.000,NaN,False,False
...,...,...,...,...,...,...
195,2025-03-02,178.71,4.193,0.285712,False,False
196,2025-03-03,142.16,4.193,0.273024,False,False
197,2025-03-04,144.79,4.195,0.265463,False,False
198,2025-03-05,146.28,4.205,0.237273,False,False


In [115]:
px.line(treasury_corr, x='date', y=f'treasury_corr_{COIN}')

In [116]:
fig = px.line(
    treasury_corr, 
    x='date', 
    y=COIN, 
    title=f'{COIN} correlation with interest rate USA', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=treasury_corr[treasury_corr[f'treasury_corr_buy_{COIN}'] == 1]['date'], 
                y=treasury_corr[treasury_corr[f'treasury_corr_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Treasury rate correlation buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=treasury_corr[treasury_corr[f'treasury_corr_sell_{COIN}'] == 1]['date'], 
                y=treasury_corr[treasury_corr[f'treasury_corr_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Treasury rate correlation sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Bollinger Bands:

In [117]:
bb = metrics.calculate_bollinger_bands(df=df, coin=COIN)
bb

,date,SOLUSDT,std_SOLUSDT,mean_SOLUSDT,top_band_SOLUSDT,bottom_band_SOLUSDT,bb_buy_SOLUSDT,bb_sell_SOLUSDT
0,2024-08-19,144.29,NaN,144.290000,NaN,NaN,0,0
1,2024-08-20,142.27,1.428356,143.280000,145.422534,141.137466,0,0
2,2024-08-21,142.79,1.048872,143.116667,144.689975,141.543358,0,0
3,2024-08-22,143.22,0.857958,143.142500,144.429437,141.855563,0,0
4,2024-08-23,153.26,4.585284,145.166000,152.043926,138.288074,0,1
...,...,...,...,...,...,...,...,...
195,2025-03-02,178.71,17.392419,161.246000,187.334629,135.157371,0,0
196,2025-03-03,142.16,16.316477,158.172000,182.646716,133.697284,0,0
197,2025-03-04,144.79,15.720935,155.989333,179.570736,132.407930,0,0
198,2025-03-05,146.28,15.448635,154.453333,177.626285,131.280382,0,0


In [118]:
fig = px.line(
    bb, 
    x='date', 
    y=COIN, 
    title=f'Bollinger Bands for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=bb[bb[f'bb_buy_{COIN}'] == 1]['date'], 
                y=bb[bb[f'bb_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Bollinger Bands buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=bb[bb[f'bb_sell_{COIN}'] == 1]['date'], 
                y=bb[bb[f'bb_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Bollinger Bands sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Mean Average Convergence Divergence (MACD):

In [119]:
macd = metrics.calculate_macd(df=df, coin=COIN)
macd

,date,SOLUSDT,EMA12_SOLUSDT,EMA26_SOLUSDT,MACD_SOLUSDT,Signal_Line_SOLUSDT,MACD_buy_SOLUSDT,MACD_sell_SOLUSDT
0,2024-08-19,144.29,144.290000,144.290000,0.000000,0.000000,0,0
1,2024-08-20,142.27,143.979231,144.140370,-0.161140,-0.032228,0,1
2,2024-08-21,142.79,143.796272,144.040343,-0.244071,-0.074596,0,0
3,2024-08-22,143.22,143.707615,143.979577,-0.271962,-0.114070,0,0
4,2024-08-23,153.26,145.177213,144.667016,0.510197,0.010784,1,0
...,...,...,...,...,...,...,...,...
195,2025-03-02,178.71,159.459827,173.828936,-14.369110,-15.144163,1,0
196,2025-03-03,142.16,156.798315,171.483089,-14.684774,-15.052285,0,0
197,2025-03-04,144.79,154.950882,169.505823,-14.554942,-14.952816,0,0
198,2025-03-05,146.28,153.616900,167.785392,-14.168492,-14.795952,0,0


In [120]:
fig = px.line(
    macd, 
    x='date', 
    y=COIN, 
    title=f'MACD for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=macd[macd[f'MACD_buy_{COIN}'] == 1]['date'], 
                y=macd[macd[f'MACD_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='MACD buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=macd[macd[f'MACD_sell_{COIN}'] == 1]['date'], 
                y=macd[macd[f'MACD_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='MACD sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Final buy setup:

In [121]:
df_buy = metrics.set_buy(df, coin=COIN)
df_buy

[*********************100%***********************]  1 of 1 completed


,date,SOLUSDT,buy_SOLUSDT,sell_SOLUSDT
0,2024-08-19,144.29,0,0
1,2024-08-20,142.27,0,0
2,2024-08-21,142.79,0,0
3,2024-08-22,143.22,0,0
4,2024-08-23,153.26,0,1
...,...,...,...,...
195,2025-03-02,178.71,0,0
196,2025-03-03,142.16,0,0
197,2025-03-04,144.79,0,0
198,2025-03-05,146.28,0,0


In [122]:
fig = px.line(
    df_buy, 
    x='date', 
    y=COIN, 
    title=f'Buy setup for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=df_buy[df_buy[f'buy_{COIN}'] == 1]['date'], 
                y=df_buy[df_buy[f'buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=df_buy[df_buy[f'sell_{COIN}'] == 1]['date'], 
                y=df_buy[df_buy[f'sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

## Evaluate models returns:

To evaluate the model return, it is possible to use a simulation of a bot working on the operations of trading for a given historic period of the coin price. Based on that, every trade will be set to operate with 10 dolars. By the end of the period, it will be computed the return of that.

Every buy will be 10 dollars
Every sell will be 10 dollars

The start value for investment will be 1000 dollars.

In [123]:
from src.simulations import simulate_model_trader

final_trader = simulate_model_trader(df=df_buy, initial_capital=1000, trade_value=10, coin=COIN)
print(f"Final balance: {final_trader['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.75548382 BTC
Coin Holdings in dollars: $112.43
Total Invested: $140.00
Final Balance: $982.43
Total Profit/Loss: $-17.57
Final balance: 982.43


### Change of strategy:

Instead of selling the BTC, try just buying when the model advises to do:

In [124]:
from src.simulations import simulate_model_buyer

final_buyer = simulate_model_buyer(df_buy, initial_capital=1000, trade_value=10, coin=COIN)
print(f"Final balance: ${final_buyer['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.80163695 BTC
Coin Holdings in dollars: $119.30
Total Invested: $140.00
Final Balance: $979.30
Total Profit/Loss: $-20.70
Final balance: $979.30


### Comparison with DCA:

In [125]:
from src.simulations import simulate_dca

final_dca = simulate_dca(df_buy, initial_capital=1000, trade_value=10, coin=COIN, investment_interval='30D')
print(f"Final balance: ${final_dca['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.39398858 BTC
Coin Holdings in dolars: $58.63
Total Invested: $70.00
Final Balance: $988.63
Total Profit/Loss: $-11.37
Final balance: $988.63


### Calculating ROI of different strategies:

$$ ROI = (\frac{\text{Final Balance} - \text{Initial Capital}}{\text{Initial Capital}})\times 100$$

In [126]:
from src.simulations import roi

roi_trader = roi(invested_amount= final_trader['total_invested'], final_balance= final_trader['coin_holdings_dollars'])
roi_buyer = roi(invested_amount = final_buyer['total_invested'], final_balance= final_buyer['coin_holdings_dollars'])
roi_dca = roi(invested_amount = final_dca['total_invested'], final_balance= final_dca['coin_holdings_dollars'])

print(f"Trader ROI: {roi_trader:.2f}%")
print(f"Buyer ROI: {roi_buyer:.2f}%")
print(f"DCA ROI: {roi_dca:.2f}%")

Trader ROI: -19.69%
Buyer ROI: -14.79%
DCA ROI: -16.24%
